In [ ]:
!pip install --upgrade coiled dask bokeh 

In [ ]:
!pip install --upgrade git+https://github.com/dask/adlfs fsspec

In [ ]:
!coiled login --token <>

In [ ]:
!pip freeze > requirements.txt

In [ ]:
import coiled

coiled.create_software_environment(
    name='banana',
    pip='requirements.txt'
)

In [1]:
%%time
import coiled 

cluster = coiled.Cluster(
    name='dkdc',
    n_workers=25,
    software='banana',
)
cluster

Creating Cluster. This takes about a minute ...Checking environment images
Valid environment image found
CPU times: user 1.32 s, sys: 268 ms, total: 1.59 s
Wall time: 3min 50s


In [2]:
from dask.distributed import Client

c = Client(cluster)
c

Client Scheduler: tls://ec2-18-218-217-0.us-east-2.compute.amazonaws.com:8786 Dashboard: https://cloud.coiled.io/dashboard/1067/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [3]:
from adlfs import AzureBlobFileSystem

storage_options = {'account_name': 'azureopendatastorage'}

fs = AzureBlobFileSystem(**storage_options)
fs.ls('isdweatherdatacontainer')

['isdweatherdatacontainer/ISDWeather/',
 'isdweatherdatacontainer/ISDWeatherAll',
 'isdweatherdatacontainer/ISDWeatherAll/',
 'isdweatherdatacontainer/ISDWeatherBackup',
 'isdweatherdatacontainer/ISDWeatherBackup/',
 'isdweatherdatacontainer/ISDWeatherBackup_20200115/',
 'isdweatherdatacontainer/ISDWeatherTest/',
 'isdweatherdatacontainer/fama',
 'isdweatherdatacontainer/fama/']

In [4]:
%%time
files = fs.glob('isdweatherdatacontainer/ISDWeather/year=*/month=*/*.parquet')
files[-5:]

CPU times: user 13.8 s, sys: 703 ms, total: 14.5 s
Wall time: 3min 54s


['isdweatherdatacontainer/ISDWeather/year=2020/month=9/part-00003-tid-284390262140683114-58f91dbe-0395-42d9-89c5-3c3192dc5f53-2443-9.c000.snappy.parquet',
 'isdweatherdatacontainer/ISDWeather/year=2020/month=9/part-00004-tid-284390262140683114-58f91dbe-0395-42d9-89c5-3c3192dc5f53-2448-9.c000.snappy.parquet',
 'isdweatherdatacontainer/ISDWeather/year=2020/month=9/part-00005-tid-284390262140683114-58f91dbe-0395-42d9-89c5-3c3192dc5f53-2445-9.c000.snappy.parquet',
 'isdweatherdatacontainer/ISDWeather/year=2020/month=9/part-00006-tid-284390262140683114-58f91dbe-0395-42d9-89c5-3c3192dc5f53-2449-9.c000.snappy.parquet',
 'isdweatherdatacontainer/ISDWeather/year=2020/month=9/part-00007-tid-284390262140683114-58f91dbe-0395-42d9-89c5-3c3192dc5f53-2444-9.c000.snappy.parquet']

In [5]:
len(files)

1224

In [6]:
files = [f'az://{file}' for file in files]
files[-5:]

['az://isdweatherdatacontainer/ISDWeather/year=2020/month=9/part-00003-tid-284390262140683114-58f91dbe-0395-42d9-89c5-3c3192dc5f53-2443-9.c000.snappy.parquet',
 'az://isdweatherdatacontainer/ISDWeather/year=2020/month=9/part-00004-tid-284390262140683114-58f91dbe-0395-42d9-89c5-3c3192dc5f53-2448-9.c000.snappy.parquet',
 'az://isdweatherdatacontainer/ISDWeather/year=2020/month=9/part-00005-tid-284390262140683114-58f91dbe-0395-42d9-89c5-3c3192dc5f53-2445-9.c000.snappy.parquet',
 'az://isdweatherdatacontainer/ISDWeather/year=2020/month=9/part-00006-tid-284390262140683114-58f91dbe-0395-42d9-89c5-3c3192dc5f53-2449-9.c000.snappy.parquet',
 'az://isdweatherdatacontainer/ISDWeather/year=2020/month=9/part-00007-tid-284390262140683114-58f91dbe-0395-42d9-89c5-3c3192dc5f53-2444-9.c000.snappy.parquet']

In [13]:
import dask.dataframe as dd 

data_url = 'az://isdweatherdatacontainer/ISDWeather/year=*/month=*/*.parquet'

columns = ['datetime', 'latitude', 'longitude', 'elevation', 'windAngle', 'windSpeed', 'temperature', 'precipTime', 'precipDepth', 'snowDepth', 'stationName', 'countryOrRegion']

df = dd.read_parquet(
    files, 
    storage_options=storage_options, 
    columns=columns,
    )
df

,datetime,latitude,longitude,elevation,windAngle,windSpeed,temperature,precipTime,precipDepth,snowDepth,stationName,countryOrRegion
npartitions=1224,,,,,,,,,,,,
,datetime64[ns],float64,float64,float64,int32,float64,float64,float64,float64,float64,object,object
,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...


In [14]:
df.head()

,datetime,latitude,longitude,elevation,windAngle,windSpeed,temperature,precipTime,precipDepth,snowDepth,stationName,countryOrRegion
0,2008-01-18 09:15:00,45.516,-103.302,879.0,NaN,NaN,-16.6,NaN,NaN,NaN,BUFFALO 13 ESE,US
1,2008-01-06 10:15:00,38.870,-100.963,875.0,NaN,NaN,1.8,NaN,NaN,NaN,OAKLEY 19 SSW,US
2,2008-01-16 18:10:00,38.870,-100.963,875.0,NaN,NaN,-6.2,NaN,NaN,NaN,OAKLEY 19 SSW,US
3,2008-01-21 18:15:00,45.516,-103.302,879.0,NaN,NaN,-17.1,NaN,NaN,NaN,BUFFALO 13 ESE,US
4,2008-01-28 00:30:00,48.306,-95.874,351.0,NaN,NaN,-0.9,NaN,NaN,NaN,GOODRIDGE 12 NNW,US


In [15]:
df = df.repartition(npartitions=128)

In [16]:
%%time
df = df.persist()
len(df)

CPU times: user 831 ms, sys: 85.1 ms, total: 916 ms
Wall time: 4min 28s


1503309322

In [17]:
%%time
len(df)

CPU times: user 212 ms, sys: 31 ms, total: 243 ms
Wall time: 1min 45s


1503309322

In [18]:
%%time 
df.describe().compute()

KilledWorker: ("('dropna-quantiles-1-d4b90a88cb8fe4d9dc8a9e3da16668e0', 89)", <Worker 'tls://10.2.12.14:45839', name: lostmygithubaccount-1067-worker-10-2fe305, memory: 0, processing: 42>)

In [12]:
c.restart()
c

Client Scheduler: tls://ec2-18-218-217-0.us-east-2.compute.amazonaws.com:8786 Dashboard: https://cloud.coiled.io/dashboard/1067/status,Cluster Workers: 24 Cores: 96 Memory: 412.32 GB


In [ ]:
# this runs fine

import dask.dataframe as dd

df = dd.read_csv(
    "s3://nyc-tlc/trip data/yellow_tripdata_2019-*.csv",
    dtype={
        "payment_type": "UInt8",
        "VendorID": "UInt8",
        "passenger_count": "UInt8",
        "RatecodeID": "UInt8",
    },
    storage_options={"anon": True},
    blocksize="16 MiB",
).persist()

df.groupby("passenger_count").tip_amount.mean().compute()